In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Charger les données
fichier = pd.read_csv("C:\\Users\\PC\\Downloads\\traitement_final_grav.csv")

# Préparer les données
X = fichier.drop(['Gravité', 'Disease'], axis=1)
y_disease = fichier['Disease']
y_gravity = fichier['Gravité']

# Encoder la gravité
encoder = LabelEncoder()
y_gravity_encoded = encoder.fit_transform(y_gravity)
y_gravity_encoded = to_categorical(y_gravity_encoded)

# Split dataset
X_train, X_test, y_train_disease, y_test_disease, y_train_gravity_encoded, y_test_gravity_encoded = train_test_split(
    X, y_disease, y_gravity_encoded, test_size=0.2, random_state=1)

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modèle pour prédire la présence d'une crise d'asthme
model_disease = Sequential([
    Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Sortie binaire
])

model_disease.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_disease.fit(X_train_scaled, y_train_disease, epochs=5, batch_size=10)
# Prédictions de la présence d'une crise d'asthme
y_pred_disease = (model_disease.predict(X_test_scaled) > 0.5).astype("int32")

# Filtrer les données pour la gravité en se basant sur les prédictions de crise d'asthme
X_train_asthma_scaled = X_train_scaled[y_train_disease == 1]
y_train_gravity_filtered = y_train_gravity_encoded[y_train_disease == 1]

# Modèle pour prédire la gravité des crises d'asthme
model_gravity = Sequential([
    Dense(64, input_dim=X_train_asthma_scaled.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(y_gravity_encoded.shape[1], activation='softmax')  # Sortie multiclasses
])

model_gravity.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_gravity.fit(X_train_asthma_scaled, y_train_gravity_filtered, epochs=20, batch_size=10)

# Filtrer les données de test pour la gravité basée sur les prédictions de crise d'asthme
X_test_asthma_scaled = X_test_scaled[y_pred_disease.flatten() == 1]
y_test_gravity_filtered = y_test_gravity_encoded[y_pred_disease.flatten() == 1]

# Prédictions de la gravité des crises d'asthme
y_pred_gravity = model_gravity.predict(X_test_asthma_scaled)
y_pred_gravity_classes = y_pred_gravity.argmax(axis=-1)

# Convertir les vraies valeurs de y_test_disease en format numpy pour la compatibilité
y_test_disease_np = y_test_disease.to_numpy()

# Matrice de confusion et rapport de classification pour la prédiction de la présence d'une crise d'asthme
print("Confusion Matrix for Disease Prediction:")
print(confusion_matrix(y_test_disease_np, y_pred_disease.flatten()))
print("\nClassification Report for Disease Prediction:")
print(classification_report(y_test_disease_np, y_pred_disease.flatten()))

# Convertir y_test_gravity_filtered en classes pour la comparaison
y_test_gravity_classes = np.argmax(y_test_gravity_filtered, axis=1)

# Matrice de confusion et rapport de classification pour la prédiction de la gravité des crises d'asthme
print("Confusion Matrix for Asthma Severity Prediction:")
print(confusion_matrix(y_test_gravity_classes, y_pred_gravity_classes))
print("\nClassification Report for Asthma Severity Prediction:")
print(classification_report(y_test_gravity_classes, y_pred_gravity_classes))


Epoch 1/5
2720/2720 [==============================] - 8s 3ms/step - loss: 0.0121 - accuracy: 0.9965
Epoch 2/5
2720/2720 [==============================] - 6s 2ms/step - loss: 0.0031 - accuracy: 0.9984
Epoch 3/5
2720/2720 [==============================] - 7s 2ms/step - loss: 0.0023 - accuracy: 0.9989
Epoch 4/5
2720/2720 [==============================] - 7s 3ms/step - loss: 0.0014 - accuracy: 0.9994
Epoch 5/5
213/213 [==============================] - 0s 2ms/step
Epoch 1/20
83/83 [==============================] - 1s 3ms/step - loss: 0.5413 - accuracy: 0.7978
Epoch 2/20
83/83 [==============================] - 0s 3ms/step - loss: 0.2515 - accuracy: 0.8904
Epoch 3/20
83/83 [==============================] - 0s 3ms/step - loss: 0.1853 - accuracy: 0.9281
Epoch 4/20
83/83 [==============================] - 0s 3ms/step - loss: 0.1549 - accuracy: 0.9306
Epoch 5/20
83/83 [==============================] - 0s 3ms/step - loss: 0.1297 - accuracy: 0.9464
Epoch 6/20
83/83 [=======================